In [1]:
import numpy as np
import pandas as pd

def calc_Rs(R1, R2, A):
    midR = (R1+R2)/2
    Ro = (A/np.pi+4*midR**2)/4/midR
    Ri = 2*midR - Ro   
    return Ri, midR, Ro

# SimpleD

In [8]:
# Rings radius
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6

AIRR1 = AF
AIRR2 = 6*AF
AIRR3 = 12*AF
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF
AORR1 = 36*AF
AORR2 = 42*AF

R = []

Ai = AIRR1
IRR1 = np.sqrt(1/np.pi*Ai)
R.append(IRR1)
print('IRR1: ', np.round(IRR1, 2))

Ai += AIRR2
IRR2 = np.sqrt(1/np.pi*Ai)
R.append(IRR2)
print('IRR2: ', np.round(IRR2, 2))

Ai += AIRR3
IRR3 = np.sqrt(1/np.pi*Ai)
R.append(IRR3)
print('IRR3: ', np.round(IRR3, 2))

Ai += AFR1
FR1 = np.sqrt(1/np.pi*Ai)
R.append(FR1)
print('FR1: ', np.round(FR1, 2))

Ai += AFR2
FR2 = np.sqrt(1/np.pi*Ai)
R.append(FR2)
print('FR2: ', np.round(FR2, 2))

Ai += AFR3
FR3 = np.sqrt(1/np.pi*Ai)
R.append(FR3)
print('FR3: ', np.round(FR3, 2))

Ai += AORR1
ORR1 = np.sqrt(1/np.pi*Ai)
R.append(ORR1)
print('ORR1: ', np.round(ORR1, 2))

Ai += AORR2
ORR2 = np.sqrt(1/np.pi*Ai)
R.append(ORR2)
print('ORR2: ', np.round(ORR2, 2))

ORR3 = 297.3
R.append(ORR3)

R = np.array(R)

IRR1:  18.9
IRR2:  50.01
IRR3:  82.39
FR1:  114.97
FR2:  147.62
FR3:  174.26
ORR1:  207.91
ORR2:  241.32


In [18]:
# Coolant channel radii - option 1
Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac
Actotal = AcR1 + AcR2 + AcR3

tj = Actotal / 2/np.pi/sum(R)
print(tj)
Rji = R - tj/2
print(Rji)
Rjo = R + tj/2
print(Rjo)

1.9303119325231488
[ 17.93619648  49.04312204  81.42392924 114.00728246 146.65912584
 173.2967032  206.94972092 240.35115772 296.33484403]
[ 19.86650841  50.97343398  83.35424117 115.93759439 148.58943777
 175.22701513 208.88003286 242.28146965 298.26515597]


In [19]:
# Coolant channel radii - option 2
Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac
Actotal = AcR1 + AcR2 + AcR3

RT = sum(R)

t = (2*RT - np.sqrt(4*RT**2 - 36*Actotal/np.pi))/18
# print(t)
Rji = R - t
print(Rji)


[ 16.95831145  48.06523701  80.4460442  113.02939742 145.68124081
 172.31881817 205.97183589 239.37327269 295.356959  ]


In [21]:
# power density averaged in the ring (divided into layers)
file = pd.read_csv('power.csv')

# W/cm3
R1Fs = np.zeros(10)
for i in range(1, 7):
    fi = np.array(file['F'+str(i)].tolist())
    R1Fs += fi/6/1e6

R2Fs = np.zeros(10)
for i in range(7, 15):
    fi = np.array(file['F'+str(i)].tolist())
    R2Fs += fi/8/1e6

R3Fs = np.zeros(10)
for i in range(15, 23):
    fi = np.array(file['F'+str(i)].tolist())
    R3Fs += fi/8/1e6

QR1s = R1Fs*AFR1/(AFR1-np.pi*(FR1**2-R[3]**2))
QR2s = R2Fs*AFR2/(AFR2-np.pi*(FR2**2-R[4]**2))
QR3s = R3Fs*AFR3/(AFR3-np.pi*(FR3**2-R[5]**2))
print('Ring one power density: ', QR1s)
print('Ring two power density: ', QR2s)
print('Ring three power density: ', QR3s)

Ring one power density:  [3.874      4.442      5.029      5.64383333 6.44416667 7.68516667
 8.68933333 9.40183333 9.07566667 6.9985    ]
Ring two power density:  [3.03225  3.558375 4.165875 4.666125 5.269125 6.34175  7.041    7.49475
 7.0175   5.1685  ]
Ring three power density:  [3.2925   3.906125 4.554625 5.1155   5.817    7.08525  7.92825  8.464125
 7.896625 5.572875]


In [22]:
# simple - film
rho = 5.7030  # [kg/m3] 6.4 MPa 259 C
cp = 5.188  # [J/g/K] 6.4 MPa 259 C [2]
cp *= 1e3  # [J/kg/K]
mu = 2.9744e-5  # Pa.s 6.4 MPa 259 C
v = 1701.71  # cm/s
v /= 100  # m/s

ri = Rji/100 # m
ro = R/100
Ah = np.pi*(ro**2-ri**2)
Ph = 2*np.pi*(ri+ro)
Dh = 4*Ah/Ph
Re = rho * v * Dh / mu
print('Re: ', Re)

k = 0.41 # W/m/K
Pr = mu * cp/k
print('Pr: ', Pr)

Nu = 0.023*(Re**0.8)*(Pr**0.4)
print('Nu: ', Nu)
h = Nu/Dh * k  # W/m^2/K
print('h: ', h)

h /= 1e4  # W/cm^2/K

ri *= 100  # cm
ro *= 100  # cm
dr = 0.01  # cm

kgci = h * (ri + dr) * np.log(1 + dr/ri)
kgco = h * (ro + dr) * np.log(1 + dr/ro)
print('ki: ', kgci)  # W/cm/C
print('ko: ', kgco)  # W/cm/C

Re:  [126794.81959681 126794.81959681 126794.81959681 126794.81959681
 126794.81959681 126794.81959681 126794.81959681 126794.81959681
 126794.81959681]
Pr:  0.37637041951219513
Nu:  [188.12877923 188.12877923 188.12877923 188.12877923 188.12877923
 188.12877923 188.12877923 188.12877923 188.12877923]
h:  [1984.84745186 1984.84745186 1984.84745186 1984.84745186 1984.84745186
 1984.84745186 1984.84745186 1984.84745186 1984.84745186]
ki:  [0.00198543 0.00198505 0.00198497 0.00198494 0.00198492 0.00198491
 0.0019849  0.00198489 0.00198488]
ko:  [0.00198537 0.00198505 0.00198497 0.00198493 0.00198491 0.0019849
 0.0019849  0.00198489 0.00198488]


# SimpleE

In [5]:
# Rings radius
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6

AIRR = 19*AF
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF

R = []

Ai = AIRR
IRR = np.sqrt(1/np.pi*Ai)
R.append(IRR)
print('IRR: ', np.round(IRR, 2))

Ai += AFR1
FR1 = np.sqrt(1/np.pi*Ai)
R.append(FR1)
print('FR1: ', np.round(FR1, 2))

Ai += AFR2
FR2 = np.sqrt(1/np.pi*Ai)
R.append(FR2)
print('FR2: ', np.round(FR2, 2))

Ai += AFR3
FR3 = np.sqrt(1/np.pi*Ai)
R.append(FR3)
print('FR3: ', np.round(FR3, 2))

ORR3 = 297.3
R.append(ORR3)

R = np.array(R)

IRR:  82.39
FR1:  114.97
FR2:  147.62
FR3:  174.26


In [10]:
Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac

print('R1:')
n = int(np.floor((FR1-IRR)/1.88))
p = (FR1-IRR)/n
R1 = [IRR]
for i in range(n-1):
    R1.append(R1[-1]+p)
print(np.round(R1, 2))
R1 = np.array(R1)
Rt = sum(R1)
t = (-2*Rt + np.sqrt(4*Rt**2+4*n*AcR1/np.pi))/2/n
print(t)
R1o = R1 + t
# print(np.round(Ro, 2))
# print(sum(np.pi*(Ro**2-R**2)))

print('R2:')
n = int(np.floor((FR2-FR1)/1.88))
# print(n)
p = (FR2-FR1)/n
# print(p)
R2 = [FR1]
for i in range(n-1):
    R2.append(R2[-1]+p)
print(np.round(R2, 2))
R2 = np.array(R2)
Rt = sum(R2)
t = (-2*Rt + np.sqrt(4*Rt**2+4*n*AcR2/np.pi))/2/n
print(t)
R2o = R2 + t
# print(np.round(Ro, 2))
# print(sum(np.pi*(Ro**2-R**2)))

print('R3:')
n = int(np.floor((FR3-FR2)/1.88))
# print(n)
p = (FR3-FR2)/n
# print(p)
R3 = [FR2]
for i in range(n-1):
    R3.append(R3[-1]+p)
print(np.round(R3, 2))
R3 = np.array(R3)
Rt = sum(R3)
t = (-2*Rt + np.sqrt(4*Rt**2+4*n*AcR3/np.pi))/2/n
print(t)
R3o = R3 + t
# print(np.round(Ro, 2))
# print(sum(np.pi*(Ro**2-R**2)))


R1:
[ 82.39  84.31  86.22  88.14  90.06  91.97  93.89  95.81  97.72  99.64
 101.56 103.47 105.39 107.31 109.22 111.14 113.06]
0.4220423255431155
R2:
[114.97 116.89 118.81 120.73 122.66 124.58 126.5  128.42 130.34 132.26
 134.18 136.1  138.02 139.94 141.86 143.78 145.7 ]
0.4221356006152532
R3:
[147.62 149.53 151.43 153.33 155.24 157.14 159.04 160.94 162.85 164.75
 166.65 168.55 170.46 172.36]
0.4177174096544474


In [3]:
# power density averaged in the ring (divided into layers)
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF

Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac


file = pd.read_csv('power.csv')

# W/cm3
R1Fs = np.zeros(10)
for i in range(1, 7):
    fi = np.array(file['F'+str(i)].tolist())
    R1Fs += fi/6/1e6

R2Fs = np.zeros(10)
for i in range(7, 15):
    fi = np.array(file['F'+str(i)].tolist())
    R2Fs += fi/8/1e6

R3Fs = np.zeros(10)
for i in range(15, 23):
    fi = np.array(file['F'+str(i)].tolist())
    R3Fs += fi/8/1e6

QR1s = R1Fs*AFR1/(AFR1-AcR1)
QR2s = R2Fs*AFR2/(AFR2-AcR2)
QR3s = R3Fs*AFR3/(AFR3-AcR3)
print('Ring one power density: ', QR1s)
print('Ring two power density: ', QR2s)
print('Ring three power density: ', QR3s)

Ring one power density:  [ 4.9573133   5.68414705  6.43529391  7.22205732  8.24619338  9.83422275
 11.11919146 12.03093274 11.61355785  8.95553876]
Ring two power density:  [3.88017895 4.55342791 5.33080732 5.97094567 6.74256671 8.11513723
 9.00992332 9.59055856 8.97985186 6.61380325]
Ring three power density:  [ 4.21320445  4.99842164  5.82826616  6.5459825   7.44364777  9.06654726
 10.14528115 10.83100657 10.10481264  7.13125642]


In [17]:
# simple - film
rho = 5.7030  # [kg/m3] 6.4 MPa 259 C
cp = 5.188  # [J/g/K] 6.4 MPa 259 C [2]
cp *= 1e3  # [J/kg/K]
mu = 2.9744e-5  # Pa.s 6.4 MPa 259 C
v = 1701.71  # cm/s
v /= 100  # m/s

Ri = np.concatenate([R1, R2, R3])
Ro = np.concatenate([R1o, R2o, R3o])

ri = Ri/100 # m
ro = Ro/100
Ah = np.pi*(ro**2-ri**2)
Ph = 2*np.pi*(ri+ro)
Dh = 4*Ah/Ph
Re = rho * v * Dh / mu
# print('Re: ', Re)

k = 0.41 # W/m/K
Pr = mu * cp/k
# print('Pr: ', Pr)

Nu = 0.023*(Re**0.8)*(Pr**0.4)
# print('Nu: ', Nu)
h = Nu/Dh * k  # W/m^2/K
# print('h: ', h)

h /= 1e4  # W/cm^2/K

ri *= 100  # cm
ro *= 100  # cm
dr = 0.01  # cm

kgci = h * (ri + dr) * np.log(1 + dr/ri)
kgco = h * (ro + dr) * np.log(1 + dr/ro)
print('ki: ', np.average(kgci))  # W/cm/C
print('ko: ', np.average(kgco))  # W/cm/C

ki:  0.0026954079929609504
ko:  0.0026954075925975027


# SimpleF

In [18]:
# Rings radius
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6

AIRR = 19*AF
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF

R = []

Ai = AIRR
IRR = np.sqrt(1/np.pi*Ai)
R.append(IRR)
print('IRR: ', np.round(IRR, 2))

Ai += AFR1
FR1 = np.sqrt(1/np.pi*Ai)
R.append(FR1)
print('FR1: ', np.round(FR1, 2))

Ai += AFR2
FR2 = np.sqrt(1/np.pi*Ai)
R.append(FR2)
print('FR2: ', np.round(FR2, 2))

Ai += AFR3
FR3 = np.sqrt(1/np.pi*Ai)
R.append(FR3)
print('FR3: ', np.round(FR3, 2))

ORR3 = 297.3
R.append(ORR3)

R = np.array(R)

IRR:  82.39
FR1:  114.97
FR2:  147.62
FR3:  174.26


In [25]:
Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac

F = np.cos(np.pi/6)*1.88*2 # min distance between cool channels
print(F)

print('R1:')
n = int(np.floor((FR1-IRR)/F))
print(n)
p = (FR1-IRR)/n
R1 = [IRR]
for i in range(n-1):
    R1.append(R1[-1]+p)
print(np.round(R1, 2))
R1 = np.array(R1)
Rt = sum(R1)
t = (-2*Rt + np.sqrt(4*Rt**2+4*n*AcR1/np.pi))/2/n
print(t)
R1o = R1 + t
# print(np.round(Ro, 2))
# print(sum(np.pi*(Ro**2-R**2)))

print('R2:')
n = int(np.floor((FR2-FR1)/F))
print(n)
p = (FR2-FR1)/n
# print(p)
R2 = [FR1]
for i in range(n-1):
    R2.append(R2[-1]+p)
print(np.round(R2, 2))
R2 = np.array(R2)
Rt = sum(R2)
t = (-2*Rt + np.sqrt(4*Rt**2+4*n*AcR2/np.pi))/2/n
print(t)
R2o = R2 + t
# print(np.round(Ro, 2))
# print(sum(np.pi*(Ro**2-R**2)))

print('R3:')
n = int(np.floor((FR3-FR2)/F))
print(n)
p = (FR3-FR2)/n
# print(p)
R3 = [FR2]
for i in range(n-1):
    R3.append(R3[-1]+p)
print(np.round(R3, 2))
R3 = np.array(R3)
Rt = sum(R3)
t = (-2*Rt + np.sqrt(4*Rt**2+4*n*AcR3/np.pi))/2/n
print(t)
R3o = R3 + t
# print(np.round(Ro, 2))
# print(sum(np.pi*(Ro**2-R**2)))


3.2562555182294894
R1:
10
[ 82.39  85.65  88.91  92.16  95.42  98.68 101.94 105.2  108.46 111.71]
0.7213107524064298
R2:
10
[114.97 118.24 121.5  124.77 128.03 131.3  134.56 137.83 141.09 144.36]
0.7205173183213901
R3:
8
[147.62 150.95 154.28 157.61 160.94 164.27 167.6  170.93]
0.7335494845636958


In [21]:
# power density averaged in the ring (divided into layers)
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF

Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac


file = pd.read_csv('power.csv')

# W/cm3
R1Fs = np.zeros(10)
for i in range(1, 7):
    fi = np.array(file['F'+str(i)].tolist())
    R1Fs += fi/6/1e6

R2Fs = np.zeros(10)
for i in range(7, 15):
    fi = np.array(file['F'+str(i)].tolist())
    R2Fs += fi/8/1e6

R3Fs = np.zeros(10)
for i in range(15, 23):
    fi = np.array(file['F'+str(i)].tolist())
    R3Fs += fi/8/1e6

QR1s = R1Fs*AFR1/(AFR1-AcR1)
QR2s = R2Fs*AFR2/(AFR2-AcR2)
QR3s = R3Fs*AFR3/(AFR3-AcR3)
print('Ring one power density: ', QR1s)
print('Ring two power density: ', QR2s)
print('Ring three power density: ', QR3s)

Ring one power density:  [ 4.9573133   5.68414705  6.43529391  7.22205732  8.24619338  9.83422275
 11.11919146 12.03093274 11.61355785  8.95553876]
Ring two power density:  [3.88017895 4.55342791 5.33080732 5.97094567 6.74256671 8.11513723
 9.00992332 9.59055856 8.97985186 6.61380325]
Ring three power density:  [ 4.21320445  4.99842164  5.82826616  6.5459825   7.44364777  9.06654726
 10.14528115 10.83100657 10.10481264  7.13125642]


In [23]:
# simple - film
rho = 5.7030  # [kg/m3] 6.4 MPa 259 C
cp = 5.188  # [J/g/K] 6.4 MPa 259 C [2]
cp *= 1e3  # [J/kg/K]
mu = 2.9744e-5  # Pa.s 6.4 MPa 259 C
v = 1701.71  # cm/s
v /= 100  # m/s

Ri = np.concatenate([R1, R2, R3])
Ro = np.concatenate([R1o, R2o, R3o])

ri = Ri/100 # m
ro = Ro/100
Ah = np.pi*(ro**2-ri**2)
Ph = 2*np.pi*(ri+ro)
Dh = 4*Ah/Ph
Re = rho * v * Dh / mu
# print('Re: ', Re)

k = 0.41 # W/m/K
Pr = mu * cp/k
# print('Pr: ', Pr)

Nu = 0.023*(Re**0.8)*(Pr**0.4)
# print('Nu: ', Nu)
h = Nu/Dh * k  # W/m^2/K
# print('h: ', h)

h /= 1e4  # W/cm^2/K

ri *= 100  # cm
ro *= 100  # cm
dr = 0.01  # cm

kgci = h * (ri + dr) * np.log(1 + dr/ri)
kgco = h * (ro + dr) * np.log(1 + dr/ro)
print('ki: ', np.average(kgci))  # W/cm/C
print('ko: ', np.average(kgco))  # W/cm/C

ki:  0.002417876793408105
ko:  0.0024178761686396532


# coupled

In [5]:
# power density averaged in the ring (divided into layers)

P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF

Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac

file = pd.read_csv('power.csv')

# W/cm3
R1Fs = np.zeros(10)
for i in range(1, 7):
    fi = np.array(file['F'+str(i)].tolist())
    R1Fs += fi/6/1e6

R2Fs = np.zeros(10)
for i in range(7, 15):
    fi = np.array(file['F'+str(i)].tolist())
    R2Fs += fi/8/1e6

R3Fs = np.zeros(10)
for i in range(15, 23):
    fi = np.array(file['F'+str(i)].tolist())
    R3Fs += fi/8/1e6

QR1s = R1Fs*AFR1/(AFR1-AcR1)
QR2s = R2Fs*AFR2/(AFR2-AcR2)
QR3s = R3Fs*AFR3/(AFR3-AcR3)

print('average: ', (np.average(QR1s) + np.average(QR2s) + np.average(QR3s))/3)

average:  7.70646919571904


In [6]:
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6

AIRR1 = AF
AIRR2 = 6*AF
AIRR3 = 12*AF
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF
AORR1 = 36*AF
AORR2 = 42*AF
R = []

Ai = AIRR1
IRR1 = np.sqrt(1/np.pi*Ai)
R.append(IRR1)
print('IRR1: ', np.round(IRR1, 2))

Ai += AIRR2
IRR2 = np.sqrt(1/np.pi*Ai)
R.append(IRR2)
print('IRR2: ', np.round(IRR2, 2))

Ai += AIRR3
IRR3 = np.sqrt(1/np.pi*Ai)
R.append(IRR3)
print('IRR3: ', np.round(IRR3, 2))

Ai += AFR1
FR1 = np.sqrt(1/np.pi*Ai)
R.append(FR1)
print('FR1: ', np.round(FR1, 2))

Ai += AFR2
FR2 = np.sqrt(1/np.pi*Ai)
R.append(FR2)
print('FR2: ', np.round(FR2, 2))

Ai += AFR3
FR3 = np.sqrt(1/np.pi*Ai)
R.append(FR3)
print('FR3: ', np.round(FR3, 2))

Ai += AORR1
ORR1 = np.sqrt(1/np.pi*Ai)
R.append(ORR1)
print('ORR1: ', np.round(ORR1, 2))

Ai += AORR2
ORR2 = np.sqrt(1/np.pi*Ai)
R.append(ORR2)
print('ORR2: ', np.round(ORR2, 2))

ORR3 = 297.3
R.append(ORR3)

R = np.array(R)

Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac

print(calc_Rs(IRR3, FR3, AcR1+AcR2+AcR3))

IRR1:  18.9
IRR2:  50.01
IRR3:  82.39
FR1:  114.97
FR2:  147.62
FR3:  174.26
ORR1:  207.91
ORR2:  241.32
(118.28707113155804, 128.325472183805, 138.36387323605197)


In [10]:
To = 598.86
Tin = 259

cp = 5188
v = 1701.71
rho = 5.7030e-6

A = np.pi*(138.36**2-118.29**2)

DT = 350e6/rho/v/cp/A
print(DT+259)


688.5766239975737


In [14]:
Af = np.pi*(FR3**2-IRR3**2)
DT*(Af-A)/Af+259

594.7341204155721

# simpleF9

In [2]:
# Rings radius
P = 18  # cm
F = P/np.cos(np.pi/6)
AF = F/2 * P * 6

AIRR = 19*AF
AFR1 = 18*AF
AFR2 = 24*AF
AFR3 = 24*AF

R = []

Ai = AIRR
IRR = np.sqrt(1/np.pi*Ai)
print('IRR: ', np.round(IRR, 2))

Ai += AFR1
Ai += AFR2
Ai += AFR3

FR3 = np.sqrt(1/np.pi*Ai)
print('FR3: ', np.round(FR3, 2))

Ac = 120*np.pi*0.794**2 + 6*np.pi*0.635**2
AcR1 = 18*Ac
AcR2 = 24*Ac
AcR3 = 24*Ac
Act = AcR1 + AcR2 + AcR3

F = np.cos(np.pi/6)*1.88*2 # min distance between cool channels
print(F)

print('Ri:')
n = int(np.floor((FR3-IRR)/F))
print(n)
p = (FR3-IRR)/n

Rm = np.linspace(IRR, FR3, n+1)
t = Act/np.pi/2/sum(Rm)
print(t)

Ri = Rm-t/2
Ri[0] = IRR
Ri[-1] = FR3-t
Ro = Ri+t
print(np.round(Ri, 3))
print(np.round(Ro, 3))

IRR:  82.39
FR3:  174.26
3.2562555182294894
Ri:
28
0.6923035208446201
[ 82.389  85.324  88.605  91.886  95.168  98.449 101.73  105.011 108.292
 111.573 114.855 118.136 121.417 124.698 127.979 131.26  134.542 137.823
 141.104 144.385 147.666 150.948 154.229 157.51  160.791 164.072 167.353
 170.635 173.57 ]
[ 83.081  86.016  89.298  92.579  95.86   99.141 102.422 105.703 108.985
 112.266 115.547 118.828 122.109 125.39  128.672 131.953 135.234 138.515
 141.796 145.077 148.359 151.64  154.921 158.202 161.483 164.764 168.046
 171.327 174.262]


In [3]:
len(Ri)

29